<a href="https://colab.research.google.com/github/bill7845/colab_repo/blob/master/dacon_%EC%A0%9C%EC%A3%BC%EC%B9%B4%EB%93%9C%EC%82%AC%EC%9A%A9%EB%9F%89%EC%98%88%EC%B8%A1/feature_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/201901-202003.csv', encoding='utf-8')
data.shape

(24697792, 12)

In [3]:
data.head(1)

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4


In [4]:
## data preprocessing

data = data.fillna('세종') # 결측값 ''로 대체

# REG_YYMM 을 datetime type 변경
# data['REG_YYMM'] = pd.to_datetime(data['REG_YYMM'],format="%Y%m")

data.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [5]:
# 결측치 확인
data.isnull().sum()

REG_YYMM        0
CARD_SIDO_NM    0
CARD_CCG_NM     0
STD_CLSS_NM     0
HOM_SIDO_NM     0
HOM_CCG_NM      0
AGE             0
SEX_CTGO_CD     0
FLC             0
CSTMR_CNT       0
AMT             0
CNT             0
dtype: int64

In [6]:
# 시군구도 필요없으니 시군구 변수도 삭제
del data['CARD_CCG_NM']
del data['HOM_CCG_NM']

In [7]:
# 칼럼명 변경
cols = ['날짜', '가맹점주소', '가맹점업종', '사용자거주시도', '연령대',
       '성별', '가구생애주기', '카드사용인원수(환불사용자포함)', '카드사용금액', '카드사용건수(환불사용자제외)',
       ]
data.columns = cols

In [8]:
# 카드사용금액을 맨 마지막 칼럼 순서로 돌리기
col_list = data.columns.tolist()
col_list.remove('카드사용금액')
col_list.insert(11,'카드사용금액')

data = data[col_list]
data.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,연령대,성별,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액
0,201901,강원,건강보조식품 소매업,강원,20s,1,1,4,4,311200
1,201901,강원,건강보조식품 소매업,강원,30s,1,2,7,8,1374500
2,201901,강원,건강보조식품 소매업,강원,30s,2,2,6,6,818700
3,201901,강원,건강보조식품 소매업,강원,40s,1,3,4,5,1717000
4,201901,강원,건강보조식품 소매업,강원,40s,1,4,3,3,1047300


In [9]:
# 성별 원핫
sex_dummies = pd.get_dummies(data['성별'],prefix='성별')
sex_dummies.columns = ['남자','여자']

data = pd.concat([data,sex_dummies],axis=1)
del data['성별']
data.head()

,날짜,가맹점주소,가맹점업종,사용자거주시도,연령대,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액,남자,여자
0,201901,강원,건강보조식품 소매업,강원,20s,1,4,4,311200,1,0
1,201901,강원,건강보조식품 소매업,강원,30s,2,7,8,1374500,1,0
2,201901,강원,건강보조식품 소매업,강원,30s,2,6,6,818700,0,1
3,201901,강원,건강보조식품 소매업,강원,40s,3,4,5,1717000,1,0
4,201901,강원,건강보조식품 소매업,강원,40s,4,3,3,1047300,1,0


In [10]:
def data_encoding(data,encoding_features): # data , ratio 변수로만들 column list

  data = data.copy() # 복사본으로 진행
  
  for store_loc in data['가맹점주소'].unique(): # 각 지역별로
    print(store_loc)
    sample_df = data.query("가맹점주소 == @store_loc") # 가맹점 주소별 샘플링

    for feature in encoding_features:

      sample_agg = sample_df.groupby(feature)['카드사용금액'].sum() # ex) 연령별 카드사용합계

      tmp_dict = dict(sample_agg) # dictionary로
      total_money = sample_agg.sum() # 카드사용금액 합계
      
      ratio = {}
      if feature == '가맹점업종': # 가맹점업종에 대해서만 인코딩 진행 시에 * 0.1
        for key in tmp_dict.keys():
          ratio[key] = round(tmp_dict[key] / total_money,5)*0.1
      else :
        for key in tmp_dict.keys():
          ratio[key] = round(tmp_dict[key] / total_money,5)
        
      data.loc[data["가맹점주소"] == store_loc, feature] = data.loc[data["가맹점주소"] == store_loc, feature].map(ratio) # 적용

  return data

In [11]:
encoding_features = ['가맹점업종','사용자거주시도','연령대','가구생애주기']
encoded_data = data_encoding(data,encoding_features)
encoded_data.insert(2,"가맹점업종_이름",data['가맹점업종'])

강원
경기
경남
경북
광주
대구
대전
부산
서울
세종
울산
인천
전남
전북
제주
충남
충북


In [27]:
encoded_data

,날짜,가맹점주소,가맹점업종_이름,가맹점업종,사용자거주시도,연령대,가구생애주기,카드사용인원수(환불사용자포함),카드사용건수(환불사용자제외),카드사용금액,남자,여자
0,201901,강원,건강보조식품 소매업,0.000115,0.7577,0.09523,0.09610,4,4,311200,1,0
1,201901,강원,건강보조식품 소매업,0.000115,0.7577,0.14925,0.16899,7,8,1374500,1,0
2,201901,강원,건강보조식품 소매업,0.000115,0.7577,0.14925,0.16899,6,6,818700,0,1
3,201901,강원,건강보조식품 소매업,0.000115,0.7577,0.24744,0.17280,4,5,1717000,1,0
4,201901,강원,건강보조식품 소매업,0.000115,0.7577,0.24744,0.34773,3,3,1047300,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,202003,충북,휴양콘도 운영업,0.000165,0.8499,0.16723,0.18210,3,4,43300,1,0
24697788,202003,충북,휴양콘도 운영업,0.000165,0.8499,0.24816,0.17579,3,3,35000,1,0
24697789,202003,충북,휴양콘도 운영업,0.000165,0.8499,0.29651,0.35239,4,6,188000,1,0
24697790,202003,충북,휴양콘도 운영업,0.000165,0.8499,0.29651,0.35239,4,6,99000,0,1
